# Laboratorio 7.2

### 2- Similitud de coseno

In [6]:

from heapq import heapify


def compute_tfidf(collection):
    # calcular los pesos TF_IDF para cada documento de la coleccion

    allWords = set([word for doc in collection for word in doc])
    N = len(collection)
    matrix = []
    for word in allWords:
        tf_docs = []
        for doc in collection:
            tf = np.log10(1 + doc.count(word))
            tf_docs.append(tf)
        df = [1 if tf > 0 else 0 for tf in tf_docs]

        df = np.log10(N / np.sum(df))
        tfidf_s = np.array(tf_docs) * df
        matrix.append(tfidf_s)
    tfidf_docs = np.array(matrix).T
    return tfidf_docs, allWords


def cosine_sim(Q, Doc):
    return np.dot(Q, Doc) / (np.linalg.norm(Q) * np.linalg.norm(Doc))


textos_tfidf, allWords = compute_tfidf(textos_procesados)

matriz = []
for doc1 in textos_tfidf:
    row = []
    for doc2 in textos_tfidf:
        row.append(cosine_sim(doc1, doc2))
    matriz.append(row)

i = 1
for row in matriz:
    print(f'Archivo{i}', [float(round(x, 2)) for x in row])
    i = i + 1


Archivo1 [1.0, 0.05, 0.04, 0.01, 0.06, 0.07]
Archivo2 [0.05, 1.0, 0.05, 0.07, 0.05, 0.04]
Archivo3 [0.04, 0.05, 1.0, 0.08, 0.03, 0.1]
Archivo4 [0.01, 0.07, 0.08, 1.0, 0.07, 0.08]
Archivo5 [0.06, 0.05, 0.03, 0.07, 1.0, 0.03]
Archivo6 [0.07, 0.04, 0.1, 0.08, 0.03, 1.0]


## P4- Similitud de coseno con el Indice Invertido

### 1- Estructura del índice invertido en Python:

In [7]:
"""
index = {
w1 : [(doc1, tf_w1_doc1), (doc3, tf_w1_doc3),(doc4, tf_w1_doc4),(doc10, tf_w1_doc10)],
w2 : [(doc1, tf_w2_doc1 ), (doc2, tf_w2_doc2)],
w3 : [(doc2, tf_w3_doc2), (doc3, tf_w3_doc3),(doc7, tf_w3_doc7)],
}

idf = {
w1 : idf_w1,
w2 : idf_w2,
w3 : idf_w3,
}

length ={
doc1: norm_doc1,
doc2: norm_doc2,
doc3: norm_doc3,
...
}
"""

'\nindex = {\nw1 : [(doc1, tf_w1_doc1), (doc3, tf_w1_doc3),(doc4, tf_w1_doc4),(doc10, tf_w1_doc10)],\nw2 : [(doc1, tf_w2_doc1 ), (doc2, tf_w2_doc2)],\nw3 : [(doc2, tf_w3_doc2), (doc3, tf_w3_doc3),(doc7, tf_w3_doc7)],\n}\n\nidf = {\nw1 : idf_w1,\nw2 : idf_w2,\nw3 : idf_w3,\n}\n\nlength ={\ndoc1: norm_doc1,\ndoc2: norm_doc2,\ndoc3: norm_doc3,\n...\n}\n'

# SPIMI 

In [87]:
import pickle
import csv
import nltk
import re
import numpy as np
from nltk.stem.snowball import SnowballStemmer
import os
import sys
import heapq

nltk.download('punkt')

class InvertIndexSPIMI:
    def __init__(self):
        # self.index_file = index_file
        # self.index = {}
        # self.idf = {}
        # self.length = {}
        self.stemmer = SnowballStemmer('spanish')
        self.stoplist = []
        self.preprocessed_words_file = "preprocessed_words.txt"
        self.spimi_file = "spimi_blocks.txt"
        self.id_block = 0
        self.free_memory_available = 500000  # 0.5 MB
        self.current_line_preprocessed_words_file = 0
        self.end_preprocessed_words_file = 0
        self.min_heap_size = 500
        self.index = "index.txt"
     
        #fd
 # TODO Process the text to create many blocks using the SPIMI algorithm
    def load_stop_list(self, filename="english_stoplist.txt"):
        with open(filename, encoding="utf-8") as file:
            self.stoplist = [line.rstrip().lower() for line in file]

    def preprocesamiento_aux(self, texto):
        words = []
        # tokenizar
        texto_tok = texto.lower()
        texto_tok = re.sub(r'[^a-zA-Z_À-ÿ]', ' ', texto)
        words = nltk.word_tokenize(texto_tok, language='english')
        # filtrar stopwords
        words = [word for word in words if word not in self.stoplist]
        # reducir palabras
        words = [self.stemmer.stem(word) for word in words]
        return words
    def get_list_tf(self, tokens, id):
        # compute the term frequency for each word in the document with id
        tf = {}
        for word in tokens:
            if word not in tf:
                tf[word] = 1
            else:
                tf[word] += 1
        return [(id, word, tf[word]) for word in tf]
        
    def preprocesamiento(self, pathOfFile):
        # only do the preprocess if the file doesnt exist
        if os.path.exists(self.preprocessed_words_file):
            return
        
        # List of tokens for each song
        preprocessed_song = []           
   
        with open(pathOfFile, newline='', encoding='utf-8') as file, open(self.preprocessed_words_file, 'w') as preprocessed_words:
            songs_file = csv.reader(file, delimiter=',', quotechar='"')
            # counter used as id for each song
            counter_rows = 0
            for row in songs_file:
                # skip the header
                if counter_rows == 0:
                    counter_rows += 1
                    continue
                # track_name , artist_name, lyrics
                song_details = row[1] + " " + row[2] + " " + row[3]
                # get list of tokens for each song
                preprocessed_song = self.preprocesamiento_aux(song_details)
                # get the tf for each word in the song
                list_tf_song = self.get_list_tf(preprocessed_song, counter_rows)
                counter_rows += 1
                # write the relative_id_song, word, tf in each line of file
                for tf in list_tf_song:
                    preprocessed_words.write(str(tf) + "\n")
    def strToList(self, string):
        string = string[:-1] 
        string = string[1:-1]
        tup = string.split(", ")
        return [int(tup[0]), tup[1], int(tup[2])]
    
    # receive the token list which is a list of tuples (id, word, tf)
    def spimi_invert(self):
        blocks_directory = r".\blocks"
        if not os.path.exists(blocks_directory):
            os.makedirs(blocks_directory)
        output_file = r".\blocks\block_" + str(self.id_block) + ".txt"
        dictionary = {}
        with open(self.preprocessed_words_file, 'r') as preprocessed_words, open(output_file, 'w') as block:
            print("In block:", self.id_block)
            preprocessed_words.seek(self.current_line_preprocessed_words_file)
            while sys.getsizeof(dictionary) < self.free_memory_available:
                # read the next token
                token = preprocessed_words.readline()
                if not token:
                    self.end_preprocessed_words_file = 1
                    break
                # convert the string to a list of relative_id_song, word and its tf
                token = self.strToList(token)
                if token[1] not in dictionary:
                    dictionary[token[1]] = [(token[0], token[2])]
                else:
                    dictionary[token[1]].append((token[0], token[2]))
            self.current_line_preprocessed_words_file = preprocessed_words.tell()
            sorted_terms = sorted(dictionary.keys())
            for term in sorted_terms:
                block.write(term + ":" + str(dictionary[term]) + "\n")
        self.id_block += 1
        
        
    def create_spimi_blocks(self): 
        # create a directory to store the blocks
        blocks_directory = r".\blocks"
        if not os.path.exists(blocks_directory):
            os.makedirs(blocks_directory)
        # read the preprocessed words file as it has the tf for each token in each song, then created size 
        # limit blocks with spimi algorithm 
        while(not self.end_preprocessed_words_file):  
            self.spimi_invert()
       
   
    """
    Para el merge se abriran todos los bloques simultaneamente, se leera la primera palabras de cada bloque,
    luego usando una pritority queue se obtendra el menor termino lexicografico y se combinaran todas las 
    listas existentes para dicho termino. Se escribira en un nuevo archivo y se repetira el proceso hasta 
    que no haya mas.
    Dado que se el minHeap se inicializa con 1 termino de cada bloque y cada vez que se hace pop, se pushea 
    un term del mismo bloque, se garantiza que el heap tiene un term de cada bloque a menos que se haya llegado
    al final de un bloque. Ademas, se itera hasta que el heap este vacio, por lo que se garantiza el 
    recorrido total por todos los bloques.
    """
   
    def merge_spimi_blocks(self):
        # get all the block names
        blocks_name = os.listdir(r".\blocks")
        # open all the blocks
        opened_blocks = []
        for block_name in blocks_name:
            opened_blocks.append(open(".\\blocks\\" + block_name, 'r'))
        # min heap for terms
        terms_from_all_blocks = []
        # read the first term from each block
        counter_opened_block = 0
        for opened_block in opened_blocks:
            term = opened_block.readline()
            if term:
                term = term.split(":")
                # push (term, [...], block_number) to the heap
                heapq.heappush(terms_from_all_blocks, (term[0], term[1], counter_opened_block))
            counter_opened_block += 1
        # merge the blocks in index.txt
        with open(self.index, 'w') as index:
            prev_top_term = None
            temp_prev_top_term = []
            # iterate until the heap is empty
            while len(terms_from_all_blocks) > 0:
                # obtenemos el menor termino lexicograficamente 
                current_top_term = heapq.heappop(terms_from_all_blocks)
                # if the previous term is the same as the current term, merge their posting lists
                if prev_top_term and prev_top_term[0] == current_top_term[0]:
                    # convert the string to a list of tuples if necessary
                    if isinstance(prev_top_term[1], str):                
                        temp_prev_top_term = eval(prev_top_term[1])
                    # posting list of current term is always a str
                    temp_current_top_term = eval(current_top_term[1])
                    # merge posting lists
                    temp_prev_top_term.extend(temp_current_top_term)
                    # update the posting list and block number 
                    prev_top_term =(prev_top_term[0], temp_prev_top_term, current_top_term[2])
                    
                else:
                    prev_top_term = current_top_term
                    # write the current term and check the type to add or no the '\n'
                    if isinstance(current_top_term[1], str):
                        index.write(current_top_term[0] + ":" + current_top_term[1])
                    elif isinstance(current_top_term[1], list):
                        index.write(current_top_term[0] + ":" + str(current_top_term[1]) + "\n")
                # read the next term from the block of the current term
                next_term_same_block = opened_blocks[current_top_term[2]].readline()
                if next_term_same_block:
                       next_term_same_block = next_term_same_block.split(":")
                       # push the next term to the heap
                       heapq.heappush(terms_from_all_blocks, (next_term_same_block[0], next_term_same_block[1], current_top_term[2]))
        # close blocks
        for opened_block in opened_blocks:
            opened_block.close()
    
        # TODO: remove all the blocks from disk. I'm not removing because reading the blocks from disk
        # is faster than processing the text again
        
        # TODO: calculate idf
                
            
            
    def build (self, pathOfFile):
        self.load_stop_list()
        self.preprocesamiento(pathOfFile)
        self.create_spimi_blocks()
        self.merge_spimi_blocks()
        
        
 #    def building(self, pathOfFile):
 #        columna_procesada = collection_text.columns[position_text]
 #        rows = collection_text.shape[0]
 #        indice = {}
 #        idf_doc = {}
 #        lenght_doc = {}
 # 
 #        # build the inverted index with the collection
 #        # compute the tf
 # 
 #        for i in range(rows):
 #            texto = collection_text[columna_procesada][i]
 #            texto = preprocesamiento(texto)
 #            for word in texto:
 #                if word not in indice:
 #                    indice[word] = [(i, 1)]
 #                else:
 #                    for j, (docx, freq) in enumerate(indice[word]):
 #                        # si la palabra ya esta en el documento no se agrega una nueva por el break
 #                        if docx == i:
 #                            indice[word][j] = (docx, freq + 1)
 #                            break
 #                    else:
 #                        indice[word].append((i, 1))
 # 
 #        # compute the idf
 #        idf_doc = {word: np.log10(rows / len(indice[word])) for word in indice}
 #        # compute the length (norm)
 #        for word in indice:
 #            for doc, tf in indice[word]:
 #                tfidf = (1 + np.log10(tf)) * idf_doc[word]
 #                if doc not in lenght_doc:
 #                    lenght_doc[doc] = tfidf ** 2
 #                else:
 #                    lenght_doc[doc] += tfidf ** 2
 #        for doc in lenght_doc:
 #            lenght_doc[doc] = np.sqrt(lenght_doc[doc])
 #        # store in disk
 #        with open(self.index_file, 'wb') as file:
 #            pickle.dump([indice, idf_doc, lenght_doc], file)
 #        #print("index constrido")
 # # FIXME 1: Adaptar todos los metodos de abajo
 #    def retrieval(self, query, k):
 #        self.load_index()
 #        # diccionario para el score
 #        score = {}
 #        # preprocesar la query: extraer los terminos unicos
 #        post_query = preprocesamiento(query)
 #        # calcular el tf-idf del query
 # 
 #        query_words = set(post_query)
 #        query_tfidf = []
 #        for wordq in query_words:
 #            if wordq in self.idf:
 #                idfq = self.idf[wordq]
 #            else:
 #                idfq = 0
 #            query_tfidf.append((1 + np.log10(post_query.count(wordq))) * idfq)
 #            # normalizar el query
 #        query_norm = np.sqrt(sum([q ** 2 for q in query_tfidf]))
 #        query_tfidf = [q / query_norm for q in query_tfidf]
 #        query_words = list(query_words)
 # 
 #        # aplicar similitud de coseno y guardarlo en el diccionario score
 #        for word in query_words:
 #            if word in self.index:
 #                idf = self.idf[word]
 #                for doc, tf in self.index[word]:
 #                    if doc not in score:
 #                        score[doc] = 0
 #                    score[doc] += query_tfidf[query_words.index(word)] * tf * idf
 #        for d in score:
 #            score[d] = score[d] / self.length[d]
 # 
 #        # ordenar el score de forma descendente
 #        result = sorted(score.items(), key=lambda tup: tup[1], reverse=True)
 #        # retornamos los k documentos mas relevantes (de mayor similitud al query)
 #        #print("resultados listos")
 # 
 #        return result[:k]
 # 
 #    def load_index(self):
 #        # load index from disk      
 #        all_dics = []
 #        try:
 #            with open(self.index_file, 'rb') as file:
 #                all_dics = pickle.load(file)
 #            self.index = all_dics[0]
 #            self.idf = all_dics[1]
 #            self.length = all_dics[2]
 #            # print("index cargado")
 #        except FileNotFoundError:
 #            with open(self.index_file, 'wb') as file:
 #                # print("index creado")
 #                pass
 # 


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [88]:
# Test preprocess songs
index = InvertIndexSPIMI()
index.build(r".\Postgres\spotify_songs.csv")

In block: 0
In block: 1
In block: 2
In block: 3
In block: 4
In block: 5
In block: 6
In block: 7
In block: 8
In block: 9
In block: 10


In [80]:
arr = "[(161, 3), (527, 16), (837, 4), (1144, 1), (1190, 18)]"
aux = eval(arr)
print(aux)

[(161, 3), (527, 16), (837, 4), (1144, 1), (1190, 18)]


### Utilizar el siguiente algoritmo. Asegúrese que la similitud de dos documentos iguales sea 1. 

<img src="image-20241004-152440.png" width="75%" align="" />

### Paso 2:	Probar el Índice

In [10]:
import pandas as pd

# Utilizar la siguiente coleccion de documentos para probar la eficiencia del indice
dataton = pd.read_csv('df_total.csv')
dataton.head()



,url,news,Type
0,https://www.larepublica.co/redirect/post/3201905,Durante el foro La banca articulador empresari...,Otra
1,https://www.larepublica.co/redirect/post/3210288,El regulador de valores de China dijo el domin...,Regulaciones
2,https://www.larepublica.co/redirect/post/3240676,En una industria históricamente masculina como...,Alianzas
3,https://www.larepublica.co/redirect/post/3342889,Con el dato de marzo el IPC interanual encaden...,Macroeconomia
4,https://www.larepublica.co/redirect/post/3427208,Ayer en Cartagena se dio inicio a la versión n...,Otra


In [11]:
def mostrarDocumentos(result):
    i = 1
    result_news = []
    ids = []
    urls = []
    scores = []
    for resuldato in result:
        # info_new = "Rank:" + str(i)+ "Url :"+ dataton["url"][resuldato[0]]+ "score: "+ str(resuldato[1])
        #result_news.append(info_new)
        ids.append(str(i))
        urls.append(dataton["url"][resuldato[0]])
        scores.append(str(resuldato[1]))
        i = i + 1
    df_news = pd.DataFrame({"ID": ids, "URL": urls, "Score": scores})
    return df_news

In [12]:


index = InvertIndex("indice11.dat")
index.load_index()
index.building(dataton, 1)  #El texto a procesar esta en la posicion 1
# 
Query1 = "El pais de China y su cooperacion"
result = index.retrieval(Query1, 10)


# Proponer 3 consultas de prueba adiconales

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b824a2a4-ec75-4e0c-9c8d-ae3f8e552746' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>

In [13]:
mostrarDocumentos(result)

,ID,URL,Score
0,1,https://www.larepublica.co/redirect/post/3210288,0.7384733783760663
1,2,https://www.larepublica.co/redirect/post/3210674,0.3765711976841568
2,3,https://www.larepublica.co/redirect/post/3272619,0.29932273809980553
3,4,https://www.larepublica.co/redirect/post/3400324,0.28182843669299396
4,5,https://www.larepublica.co/redirect/post/3268483,0.25770113961402363
5,6,https://www.larepublica.co/redirect/post/3202517,0.22445755100245576
6,7,https://www.larepublica.co/redirect/post/3294146,0.22413957361890074
7,8,https://www.larepublica.co/redirect/post/3405368,0.20006317449382055
8,9,https://www.larepublica.co/redirect/post/3375722,0.19922673830097193
9,10,https://www.larepublica.co/redirect/post/2963453,0.18592112008076767


## Consultas adicionales

In [14]:
Query2 = "Empresas mas sostenibles del mundo"
result2 = index.retrieval(Query2, 5)
mostrarDocumentos(result2)

,ID,URL,Score
0,1,https://www.bbva.com/es/sostenibilidad/bbva-ot...,0.4005098937746341
1,2,https://www.bbva.com/es/sostenibilidad/que-es-...,0.37965433294693923
2,3,https://www.bbva.com/es/sostenibilidad/banca-p...,0.3665823694656286
3,4,https://www.bbva.com/es/sostenibilidad/bbva-el...,0.33341185357384023
4,5,https://www.bbva.com/es/mx/sostenibilidad/las-...,0.33161216878346766


In [15]:
Query3 = "Uso de la tecnologia en la educacion"
result3 = index.retrieval(Query3, 5)
mostrarDocumentos(result3)

,ID,URL,Score
0,1,https://www.bbva.com/es/sostenibilidad/mi-hija...,1.0707990573138608
1,2,https://www.bbva.com/es/sostenibilidad/cuatro-...,0.6787281394465713
2,3,https://www.larepublica.co/redirect/post/3190074,0.6484372042404604
3,4,https://www.larepublica.co/redirect/post/3238772,0.5496885615670838
4,5,https://www.larepublica.co/redirect/post/3461399,0.42930244739174467


In [16]:
Query4 = "Salud y bienestar en la sociedad"
result4 = index.retrieval(Query4, 10)
mostrarDocumentos(result4)

,ID,URL,Score
0,1,https://www.bbva.com/es/bienestar-psicologico-...,0.46765076194006755
1,2,https://www.bbva.com/es/es/el-crecimiento-de-l...,0.2716554022237061
2,3,https://www.bbva.com/es/diagnosticos-precoces-...,0.24011456999499284
3,4,https://www.larepublica.co/redirect/post/3236895,0.23928459884318612
4,5,https://www.bbva.com/es/bbva-y-barcelona-healt...,0.1842586905019784
5,6,https://www.larepublica.co/redirect/post/3403198,0.16651893486089497
6,7,https://www.bbva.com/es/sostenibilidad/aumenta...,0.1534335849189078
7,8,https://www.bbva.com/es/sostenibilidad/aumenta...,0.1534335849189078
8,9,https://www.bbva.com/es/sostenibilidad/aumenta...,0.1534335849189078
9,10,https://www.larepublica.co/redirect/post/3314390,0.1446902152812807


In [17]:
def wrapper(query, k):
    result = index.retrieval(query, k)
    return mostrarDocumentos(result)

In [18]:
print(wrapper("El pais de China y su cooperacion", 5))

  ID                                               URL                Score
0  1  https://www.larepublica.co/redirect/post/3210288   0.7384733783760663
1  2  https://www.larepublica.co/redirect/post/3210674   0.3765711976841568
2  3  https://www.larepublica.co/redirect/post/3272619  0.29932273809980553
3  4  https://www.larepublica.co/redirect/post/3400324  0.28182843669299396
4  5  https://www.larepublica.co/redirect/post/3268483  0.25770113961402363


In [19]:
import gradio as gr

demo = gr.Interface(
    fn=wrapper,
    inputs=[gr.Text(label="Query"), gr.Slider(label="Top Scores")],
    outputs=[gr.Dataframe(
        headers=["ID", "Url", "Score"],
        datatype=["str", "str", "str"],
    )],
    title="Proyecto DB2",
    examples=[
        ["El pais de China y su cooperacion", 5],
        ["Salud y bienestar en la sociedad", 10],
    ],
    theme='HaleyCH/HaleyCH_Theme'
)



In [21]:
# demo.close()


Closing server running on port: 7860


In [20]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
